# Machine Translation using Noisy Channel

Here we will attempt to translate old Irish to new Irish using Noisy Channel.


*   We ingest our data and concatenate the words into sentences
*   We calculate frequencies of each source word per sentence and repeat for the target words
*   We also compute frequencies of a target word being matched by the adjacent word in the target sentence for all occurences of the source word
*   Use Bayesian probabilities to predict the next word when translating test data: 
  **P(new|old) = P(new)P(old|new)**

* We process stop words, auxililary verbs and punctuations etc separately as they are more frequent than most words and thus dominate probability calculations

  **Limitations**

- Cuts off rest of target sentence once end of source sentence reached

## Read Files

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np


path_train_source = r"/content/train-source.txt"
path_train_target = r"/content/train-target.txt"

In [5]:
def read_file(path):

  text = []
  words = set()

  sentences = open(path, 'r', encoding='utf-8')

  sent = ""
  
  for ind, word in enumerate(sentences):

    token = word.rstrip("\n").rstrip("\t").lower()

    if token == "<s>" or token in ["'", '"']:
      # sent = sent
      continue

    elif token == '</s>':
      sent = sent[:-1]
      text.append(sent.lstrip())
      sent = ""

    else:
      sent = sent + token + " "    
  
    if token not in words:
      words.add(token)

  word_index = dict([(i, word) for i, word in enumerate(words)])

  return text, words, word_index

In [ ]:
%%shell
pip install unidecode

In [128]:
from difflib import SequenceMatcher as SM
from unidecode import unidecode as un_dec

def compute_frequencies(source, target):

  target_frequencies = {}
  source_to_target_frequencies = {}
  total_st_frequencies = 0

  count = 0

  for x in zip(source, target):

    source_words = [word for word in x[0].split(" ")]
    target_words = [word for word in x[1].split(" ")]

    for ind, s_w in enumerate(source_words):

      if ind <= len(target_words) - 1:

        t_w = target_words[ind]
        s_normed = un_dec(s_w)
        t_normed = un_dec(t_w)

        # compare similarity of the two words
        # if not match criteria, completely ignore
        if not (SM(None, s_normed, t_normed).ratio() > 0.6 and len(min(s_w,t_w))/len(max(s_w,t_w)) > 0.6):
          t_w = compare_words(s_w, t_w, target_words[ind:])
          count+=1
        else:

          # Add target to emission table
          if t_w in target_frequencies.keys():
            target_frequencies[t_w] += 1
          else:
            target_frequencies[t_w] = 1

          # Add target to transition table for source words
          if s_w in source_to_target_frequencies.keys():
            if t_w in source_to_target_frequencies[s_w].keys():
              source_to_target_frequencies[s_w][t_w] += 1
            else:
              source_to_target_frequencies[s_w][t_w] = 1

          else:
            source_to_target_frequencies[s_w] ={}          
            source_to_target_frequencies[s_w][t_w] = 1

          total_st_frequencies += 1

  return target_frequencies, source_to_target_frequencies, total_st_frequencies

In [90]:
def compare_words(source, target, target_list, ind = -1):

  s_normed = un_dec(source)
  t_normed = un_dec(target)

  ind += 1

  if ind > len(target_list)-1:
    return "<PAD>"

  else:

    if SM(None, s_normed, t_normed).ratio() > 0.4 and len(min(source, target))/len(max(source, target)) > 0.4:
      return target

    else:
      target = target_list[ind]
      return compare_words(source, target, target_list, ind)


In [91]:
input_text, input_words, input_word_index = read_file(path_train_source)
target_text, target_words, target_word_index = read_file(path_train_target)

input_words.add("<PAD>")
target_words.add("<PAD>")

input_word_index[len(input_word_index)] = "<PAD>"
target_word_index[len(target_word_index)] = "<PAD>"


In [130]:
target_frequencies, source_to_target_frequencies, total_st_frequencies = compute_frequencies(input_text, target_text)

## Noisy Channel Model



### Model Definition

In [ ]:
%%shell
pip install advertools

In [134]:
import advertools as adv
def predict(source_sentence, target_words, target_word_index, target_frequencies, source_to_target_frequencies):  

  target = []

  p_s = [",", "!", "-", ".", "?","'", '"',"á", "na", "liom", "leis", "léi", "'un", "<PAD>"]
  stop_punct = list(adv.stopwords["irish"])
  stop_punct = stop_punct + p_s

  total_freqs = sum(target_frequencies.values())

  for ind, val in enumerate(source_sentence):

    # if test source word is not in train set, we assume transformed to empty string and skip
    if val not in source_to_target_frequencies.keys(): 
      continue

    probs = {}

    try:
      total_tran_freq = sum(source_to_target_frequencies[val].values())      
      for x in source_to_target_frequencies[val].keys():
        
        # handle punctuation and stop words separately
        # they have highest frequencies hence risk of blowing up on prob scores

        if (x in stop_punct and val not in stop_punct):
          x_prob = 0.0
        else:
          
          # find "emission" probabilities
          try:
            x_prior = target_frequencies[x]/total_freqs
          except KeyError:
            x_prior = 0.0

          # find "transition" probabilities
          try:
            x_post = source_to_target_frequencies[val][x] / total_tran_freq
          except KeyError:
            x_post = 0.0

          x_prob = x_prior * x_post
  
        probs[x] = x_prob

    except KeyError:
      x_prob = 0.0
      probs[x] = x_prob

    new_word = max(probs, key=probs.get)

    # equate <PAD> to empty string
    if new_word == "<PAD>":
      continue

    target.append(new_word)

  return target




### Evaluate

In [95]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

def evaluate(target_words, target_word_index, target_frequencies, source_to_target_frequencies):

  testsource = open('test-source.txt', 'r')
  original = []
  sentence = []
  from_scratch_hypotheses = []
  

  for line in testsource:
    token = line.rstrip("\n").lower()
    if token=='<s>':
      sentence = []
    elif token == '</s>':
      from_scratch_hypotheses.append(predict(sentence,target_words, target_word_index, target_frequencies, source_to_target_frequencies))
      original.append(sentence)
    else:
      sentence.append(token)

  references = []
  testtarget = open('test-target.txt', 'r')

  for line in testtarget:
    token = line.rstrip("\n").lower()
    if token=='<s>':
      sentence = []
    elif token == '</s>':
      references.append([sentence])
    else: 
      sentence.append(token)

  return print((corpus_bleu(references,from_scratch_hypotheses))), from_scratch_hypotheses, references,original

In [131]:
evaluate(target_words, target_word_index, target_frequencies, source_to_target_frequencies)[0]

0.7301463927393244


## Optimization

### Build N-Grams

In [108]:
def build_ngrams(input_corpus, target_corpus, len_gram):

  ngram_counts = {}
  gram_vocab_counts = {}
  gram_vocab_counts['<PAD>-<PAD>'] = len_gram - 1

  x_ind = 0
  for x in zip(input_corpus, target_corpus):
    x_ind += 1

    source_words = [word for word in x[0].split(" ")]
    target_words = [word for word in x[1].split(" ")]

    # skip sentence pairs whose length varies by a certain length:
    if abs(len(source_words)-len(target_words)) > 5:
      continue

    s_w = "<PAD>" #initialize source_word
    t_ind = 0 #target index
    t_w = target_words[t_ind] #initialize source_word
    s_normed = un_dec(source_words[0])
    t_normed = un_dec(t_w)

    if not (SM(None, s_normed, t_normed).ratio() > 0.4 and len(min(s_w,t_w))/len(max(s_w,t_w)) > 0.4):
      t_w = compare_words(s_w, t_w, target_words[0:])
    else:
      ngram_key = s_w +"-" + source_words[0]

      # set first target as output word of ngram of <PAD> + first source word
      ngram_counts, gram_vocab_counts = update_ngram_counts(ngram_key, t_w, ngram_counts, gram_vocab_counts)

      # Sliding through corpus to get ngram counts
      for i in range(len(source_words) - (len_gram-1)):
        t_ind += 1
        i += 1
        end = i + len_gram

        # if source sentence is longer than target sentence and source index exceeds
        # target length, skip target index backwards

        if i >= len(target_words):
          t_ind = t_ind - 1

        # get next source word
        s_w = source_words[i]
        t_w = target_words[t_ind]

        # normalize source and target words and compare similarity
        # if not similar, recursively compare next words in the target sentence
        s_normed = un_dec(s_w)
        t_normed = un_dec(t_w)

        if not (SM(None, s_normed, t_normed).ratio() > 0.4 and len(min(s_w,t_w))/len(max(s_w,t_w)) > 0.4):
          t_w = compare_words(s_w, t_w, target_words[i:])
          t_ind -= 1

        # Getting ngram
        ngram = [source_words[ind] for ind in range(i-1, i+1)]
        ngram_key = "-".join(word for word in ngram)

        ngram_counts, gram_vocab_counts = update_ngram_counts(ngram_key, t_w, ngram_counts, gram_vocab_counts)


  return ngram_counts, gram_vocab_counts, gram_vocab_counts.keys()
      

In [14]:
def update_ngram_counts(key, t_w, ngram_counts, gram_vocab_counts):

  # Keeping track of the ngram counts
  if key in ngram_counts.keys():
      if t_w in ngram_counts[key]:
        ngram_counts[key][t_w] += 1
      else:
        ngram_counts[key][t_w] = 1
  else:
      ngram_counts[key] = {}
      ngram_counts[key][t_w] = 1

  # Track vocab counts
  if key in gram_vocab_counts.keys():
    gram_vocab_counts[key] += 1
  else:
    gram_vocab_counts[key] = 1

  return ngram_counts, gram_vocab_counts


In [110]:
ngram_to_target_counts, ngram_vocab_counts, ngram_vocab = build_ngrams(input_text, target_text, 2)

### Model Definition

In [102]:
def suggest_next(input, ngram_counts, vocab_counts, vocab, target_words, len_gram,  laplace_weight=0):

  target = []

  p_s = [",", "!", "-", ".", "?","'", '"',"á", "na", "liom", "leis", "léi", "'un", "<PAD>"]
  stop_punct = list(adv.stopwords["irish"])
  stop_punct = stop_punct + p_s


  if isinstance(input, str):
    source_sentence = input.split(" ")
  else: 
    source_sentence = input

  total_freqs = sum(vocab_counts.values())

  pad_str = "<PAD>-" * (len_gram-1)
  s_w = source_sentence[0]
  source_gram = pad_str + s_w 

  if source_gram in ngram_counts.keys():

    vocab_probabilities = {}
    total_tran_freq = sum(ngram_counts[source_gram].values())

    for x in ngram_counts[source_gram].keys():

      if (x in stop_punct and s_w not in stop_punct):
        x_prob = 0.0
      else:
        x_prob = calc_probs(vocab_counts[source_gram], total_freqs, ngram_counts[source_gram][x], total_tran_freq)    

      vocab_probabilities[x] = x_prob

    new_word = max(vocab_probabilities, key=vocab_probabilities.get)

    if new_word != "<PAD>":
      target.append(new_word)

    

  for ind, val in enumerate(source_sentence):

    if ind < len(source_sentence) - 1:
      vocab_probabilities = {}
      source_gram = source_sentence[ind] + "-" + source_sentence[ind+1]

      # if test source ngram is not in train set, we assume transformed to empty string and skip
      if source_gram not in ngram_counts.keys():
        continue

      total_tran_freq = sum(ngram_counts[source_gram].values())

      for x in ngram_counts[source_gram].keys():

        if (x in stop_punct and s_w not in stop_punct):
          x_prob = 0.0
        else:
          x_prob = calc_probs(vocab_counts[source_gram], total_freqs, ngram_counts[source_gram][x], total_tran_freq)    

        vocab_probabilities[x] = x_prob

      new_word = max(vocab_probabilities, key=vocab_probabilities.get)

      if new_word != "<PAD>":
        target.append(new_word)


  return target

In [103]:
def calc_probs(ngram_freq, total_freqs, word_tran_freq, total_tran_freq):

  # find "emission" probabilities
  try:
    x_prior = ngram_freq / total_freqs

  except KeyError:
    x_prior = 0.0

  # find "transition" probabilities
  try:
    x_post = word_tran_freq / total_tran_freq
  except KeyError:
    x_post = 0.0

  x_prob = x_prior * x_post

  return x_prob

In [87]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

def evaluate_alt(target_words, ngram_to_target_counts, ngram_vocab_counts, ngram_vocab):

  testsource = open('test-source.txt', 'r')
  original = []
  sentence = []
  from_scratch_hypotheses = []
  

  for line in testsource:
    token = line.rstrip("\n").lower()

    if token=='<s>':
      sentence = []

    elif token == '</s>':
      from_scratch_hypotheses.append(suggest_next(sentence,
                                                  ngram_to_target_counts,
                                                  ngram_vocab_counts, 
                                                  ngram_vocab, 
                                                  target_words, 
                                                  len_gram=2))
      original.append(sentence)
    else:
      sentence.append(token)

  references = []

  testtarget = open('test-target.txt', 'r')

  for line in testtarget:
    token = line.rstrip("\n").lower()
    if token=='<s>':
      sentence = []
    elif token == '</s>':
      references.append([sentence])
    else: 
      sentence.append(token)
      
  return print((corpus_bleu(references,from_scratch_hypotheses))), from_scratch_hypotheses, references,original

In [111]:
_, pred, ref, original = evaluate_alt(target_words, ngram_to_target_counts, ngram_vocab_counts, ngram_vocab)

0.30349414948351977
